# Exercises03

## exercise 3.1. 


If $x = \sqrt c $ then $x^2 - c = 0$.

- a) Use this root condition to construct a Newton's method for determining the
square root that uses only simple arithmetic operations (addition, subtraction,
multiplication and division).


- b) Given an arbitrary value of c > 0, how would you find a starting value to
begin Newton's method?


- c) Write a python function

      function x=newtroot(c)

that implements the method. The procedure should be self-contained (i.e., it
should not call a generic root-finding algorithm).



In [1]:
def newtroot(c):
    x = c / 2 # a starting value to begin Newton's method
    for it in range(150):
        fx = x ** 2 - c
        if abs(fx) < 1.e-9:
            break
        else:
            x = x - fx / (2 * x) # 2*x is the jacobian

    return x

# testing with some known values
for c in [1.0, 25.0, 49.0, 30.25, 100]:
    print('newtroot({:g}) = {:g}'.format(c, newtroot(c)))

newtroot(1) = 1
newtroot(25) = 5
newtroot(49) = 7
newtroot(30.25) = 5.5
newtroot(100) = 10


##  exercise 3.2
The computation of $ \sqrt {1+c^2} - 1$ can fail due to **overflow or underflow**: 

when $c$ is large, squaring it can exceed the largest representable number (realmax? in Python), whereas when c is small, the addition $1 + c^2$ will be truncated to 1.

Noting that $x = \sqrt{1 + c^2} - 1$ is equivalent to the condition

$$(x + 1)^2 - (1 + c^2) = 0$$

    f(x) = (x+1)^2 - (1+c^2)

Determine the iterations of the Newton method for finding x and a good starting
value for the iterations. 

Write a Python program that accepts c and returns
x, using only simple arithmetic operations (i.e., do not use power, log, square
root operators). The procedure should be self-contained (i.e., it should not call
a generic root-finding algorithm). Be sure to deal with the **overflow problem**.




In [9]:
def newtroot2(c):
    x = abs(c) # starting value
    for it in range(150):
        step = (1.0 / (2.0 * x + 2.0)) * ((x + 1) ** 2 - (1 + c ** 2)) # jacobian*f(x)
        if abs(step) < 1.e-9:
            break
        else:
            x =x - step

    return x

# testing with some extreme values, not pass
for c in [0.0, 1.0, 1.e-12, 1.e250]:
    print('newtroot2({:g}) = {:g}'.format(c, newtroot2(c)))

newtroot2(0) = 0
newtroot2(1) = 0.414214
newtroot2(1e-12) = 1e-12


OverflowError: (34, 'Numerical result out of range')

    (1.0 / (2.0 * x + 2.0)) * ((x + 1) ** 2 - (1 + c ** 2))

$$\frac{1}{2*( x +1)} * ((x + 1) ^ 2 - (1 + c^ 2))$$


whereas when c is small, the addition $1 + c^2$ will be truncated to 1.

    step = (x + 1 - ((1 + c) / (1 + x)) * (1 + c) + 2 * (c / (1 + x))) / 2 # trick ?

$$\frac{x + 1 - \frac{1 + c}{ 1 + x} * (1 + c) + 2 * \frac{c}{(1 + x)}}{2}$$





$$ \frac{1}{2*(x+1) } *  ( (x+1)^2  - (1+c)^2 + 2*c     )  $$


 
The addition $(1 + c)^2 - 2c$ will not be problem anymore.
  

In [2]:
def newtroot2(c):
    x = abs(c) # starting value
    for it in range(150):
        step = (x + 1 - ((1 + c) / (1 + x)) * (1 + c) + 2 * (c / (1 + x))) / 2 # trick ?
        if abs(step) < 1.e-9:
            break
        else:
            x =x - step

    return x

# testing with some extreme values
for c in [0.0, 1.0, 1.e-12, 1.e250]:
    print('newtroot2({:g}) = {:g}'.format(c, newtroot2(c)))

newtroot2(0) = 0
newtroot2(1) = 0.414214
newtroot2(1e-12) = 1e-12
newtroot2(1e+250) = 1e+250


## exercise 3.3

### Black-Scholes Option Pricing Formula

The Black-Scholes option pricing formula expresses the value of an option as a
function of the current value of the underlying asset, S, the option's strike price
K, the time-to-maturity on the option,  , the current risk-free interest rate, r,
a dividend rate, Æ, and the volatility of the the price of the underlying asset, .

In [7]:
import numpy as np
from numpy import log, exp, sqrt

from math import fabs
from warnings import warn

from scipy import stats




In [11]:
Phi = stats.norm.cdf


def BSVal(S, K, tau, r, delta, sigma):
    edtS = exp(-delta * tau) * S
    ertK = exp(-r* tau) * K
    sigmat = sigma * sqrt(tau)

    d = (log(edtS) - log(ertK)) / sigmat + sigmat / 2
    value = edtS * Phi(d) - ertK * Phi(d - sigmat)
    value_sigma = edtS * sqrt(tau / (2 * np.pi)) * exp(-0.5 * d ** 2)
    return value, value_sigma

def ImpVol(S, K, tau, r, delta, V):
    sigma = 1.0
    for it in range(150):
        value, dvalue = BSVal(S, K, tau, r, delta, sigma)
        f = V - value
        if abs(f) < 1.e-9:
            break
        else:
            step = (V - value) / dvalue
            sigma += step

    return sigma

# Testing the formula with parameters from demfin02
S, K, tau, r, delta = 1.12, 1.0, 1, 0.05, 0.03
sigma = 0.2

market_value = BSVal(S, K, tau, r, delta, sigma)[0]
implied_sigma = ImpVol(S, K, tau, r, delta, market_value)
print('original sigma = ', sigma)
print('implied sigma = ', implied_sigma)
print('market value = ', market_value)

original sigma =  0.2
implied sigma =  0.200000000007
market value =  0.166317210914


## exercise 3.5 

Consider the function $f : R^2 -> R^2  $  defined by



    fval = [200 * x * (y - x ** 2) - x + 1, 100 * (x ** 2 - y)]
    
    fjac = [[200 * (y - 3*x**2) - 1, 200*x],
            [200 * x, -100]]

Write a Python function 'func' that takes a column 2-vector x as input and
returns f, a column 2-vector that contains the value of f at x, and d, a 2 by 2
matrix that contains the Jacobian of f at x.

(a) Compute numerically the root of f via Newton's method.

(b) Compute numerically the root of f via Broyden's method.


In [16]:
def func(z):
    x, y = z
    fval = [200 * x * (y - x ** 2) - x + 1, 100 * (x ** 2 - y)]
    fjac = [[200 * (y - 3*x**2) - 1, 200*x],
            [200 * x, -100]]
    return np.array(fval), np.array(fjac)



In [17]:
def mynewton(f, x0, maxit=1000, tol=1/1000 ):
    x = x0
    for it in np.arange(maxit):
        fval, fjac = f(x)
        x = x - np.linalg.inv(fjac).dot(fval)
        if np.linalg.norm(fval)<tol:
            break
    return x    

In [18]:
x0 = np.array([0.4, 1.4])



print('Solution by Newton:', mynewton(func,x0))






Solution by Newton: [ 1.          0.99999951]


In [41]:
f = lambda x: np.array([200 * x[0] * (x[1] - x[0] ** 2) - x[0] + 1, 100 * (x[0] ** 2 - x[1])])

In [42]:
from scipy.optimize import fsolve
fsolve(func = f, x0 = x0, xtol=1/ 1000)  # without jacobian

array([ 0.99999477,  0.99998955])

In [ ]:
#https://github.com/randall-romero/CompEcon-python/blob/master/compecon/tools.py
# The script also computes an initial guess for the inverse Jacobian by inverting the finite difierence derivative computed
# using the toolbox function fdjac, which is discussed in Chapter 5 (page 107).
def jacobian(func, x, *args, **kwargs):

    # if type(func(x, *args, **kwargs)) is tuple:
    #     F = lambda x: func(x, *args, **kwargs)[0]
    # else:
    #     F = lambda x: func(x, *args, **kwargs)
    F = lambda z: func(z, *args, **kwargs)

    x = x.flatten()
    dx = x.size
    f = F(x)
    df = f.size
    x = x.astype(float)

    ''' Compute Jacobian'''
    tol = np.spacing(1) ** (1/3)

    h = tol * np.maximum(abs(x), 1)
    x_minus_h = x - h
    x_plus_h = x + h
    deltaX = x_plus_h - x_minus_h
    fx = np.zeros((dx, df))

    for k in range(dx):
        xx = x.copy()
        xx[k] = x_plus_h[k]
        fplus = F(xx)

        xx[k] = x_minus_h[k]
        fminus = F(xx)

        fx[k] = np.squeeze((fplus - fminus) / deltaX[k])  # fixme doing this to deal with broadcasting

    return fx.T


In [69]:
def mybroyden(f, x0, maxit = 1000, tol = 1/1000):
    
    x=x0
    A = f(x)
    _is_there_jacobian = (type(A) is tuple) and (len(A) == 2)
    
    if _is_there_jacobian:
        print('Jacobian was  provided by user!')
        fval,fjac = f(x)
    else:    
        print('Jacobian was not provided by user!')
        fval = f(x)
        try:
            fjac = jacobian(f,x)
        except NameError:
            print("jacobian function Not in scope!\n Using identity matrix as jacobian matrix")
            fjac = np.identity(x.size)
        else:
            print("jacobian function In scope!")
            #fjac = jacobian(f,x)  
#         return None
#     fval = f(x)[0] if _is_there_jacobian else f(x)
#     fjac = f(x)[1] if _is_there_jacobian else jacobian(f,x)   # take  one output from f(x)
    # using numpy to calculate inversed Jacobian, only for initialize. later, we update it without calculation of new inversed Jacobian.
    fjacinv = np.linalg.pinv(np.atleast_2d(fjac))
    # or we can use identity matrix as an initiative matrix
    #fjacinv = - np.identity(x.size)
    for it in range(maxit):
        fnorm = np.linalg.norm(fval)
        if fnorm<tol: 
            break 
        d = -(fjacinv.dot(fval))
        # update x
        x = x+d
        fold = fval
        fval = f(x)[0] if _is_there_jacobian else f(x)# two outputs
        # update jacobian
        u = fjacinv.dot((fval-fold))
        fjacinv = fjacinv + np.outer((d-u), np.dot(d.T, fjacinv))/np.dot(d.T,u) ## ? np.outer !!! key
    return x   


In [70]:
print('Solution by Broyden:', mybroyden(func,x0, maxit = 10000, tol = 1/10000))

Jacobian was  provided by user!
Solution by Broyden: [ 0.99998571  0.99997149]


In [29]:
from scipy.optimize import broyden1

broyden1(f,x0, iter = 10000 )

array([ 1.,  1.])

##  exercise 3.6

common problem in computation is finding the inverse of a cumulative distribution
function (CDF). 

A CDF is a function, F, that is nondecreasing over
some domain [a; b] and for which F(a) = 0 and F(b) = 1. 



Write a function that
uses Newton's method to solve inverse CDF problems. The function should
take the following form:

    x=icdf(p,F,x0,varargin)


where p is a probability value (a real number on [0,1]), F is the name of a
function file, and x0 is a starting value for the Newton iterations.

In [8]:

def icdf(p, F, x0, *args):
    x = x0
    for it in range(150):
        cdf, pdf = F(x, *args)
        if abs(cdf - p) < 1.e-9:
            break
        else:
            x += (p - cdf) / pdf

    return x

# testing with a normal distribution
phi = stats.norm.pdf
Phi = stats.norm.cdf

def cdfnormal(x, mu, sigma):
    z = (x-mu) / sigma
    return Phi(z), phi(z)

for p in [0.01, 0.05, 0.5, 0.95, 0.99]:
    print('icdf({:g}) = {:g}'.format(p, icdf(p, cdfnormal,0.0,0.0,1.0)))

icdf(0.01) = -2.32635
icdf(0.05) = -1.64485
icdf(0.5) = 0
icdf(0.95) = 1.64485
icdf(0.99) = 2.32635


In [4]:

def icdf(p, func, a, b, tol=1 * 10 ** -6, iterations=100):
    """
    Calculate quantile func using the bisection method
    """
    x = (a + b) / 2.0
    d = (b - a) / 2.0
    for _ in range(iterations):
        result = func(x) - p
        d = d / 2
        if fabs(result) < tol:
            return x
        elif result > 0:
            x = x - d
        else:
            x = x + d
    else:
        warn("We were unable to find a value suitably close to {} after {} iterations"
             "".format(p, iterations))
        return x




In [5]:
icdf(.6, stats.norm.cdf, -10, 10)

0.25334835052490234